In [52]:
import pandas as pd
import psycopg2
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline


SENTIMENTS = {
        'POSITIVE': 1,
        'NEGATIVE': -1,
        'NEUTRAL': 0,
        'UNKNOWN': 2
    }

CREDENTIALS = {
    'host': 'ec2-50-17-207-16.compute-1.amazonaws.com',
    'database': 'XXX',
    'user': 'XXX',
    'port': '5432',
    'pwd': 'XXXXX',
    'uri': 'postgres://XXX@host:5432/XX',
    'cli': 'heroku pg:psql postgresql-flat-49197 --app beepper'
}

COLUMNS = ['id','txt']

try:
    conn = psycopg2.connect(
        database=CREDENTIALS.get('database'),
        user=CREDENTIALS.get('user'),
        password=CREDENTIALS.get('pwd'),
        host=CREDENTIALS.get('host'),
        port=CREDENTIALS.get('port'),
    )
except Exception as e:
    print("Connection to database failed")

curr = conn.cursor()
curr.execute("SELECT id, txt_clean FROM hello_question")
res = curr.fetchall()

curr.close()
conn.close()

In [8]:
data = pd.DataFrame(res,columns=COLUMNS).reset_index(drop=True)
# data.to_csv('data/extract_23_02.csv',index=False)

# add labels

In [74]:
groups = {'médicaments': [1012, 994, 974, 919, 913, 801, 776, 770, 760, 733],
'administratif': [1015, 1013, 1011, 1009, 986, 984, 981, 980, 979, 918],
'pratique médicale': [1001,891,942,934,965,926,968,1000,895,864], 
'diagnostique' : [989,987,985,964,957,951,950,937,903,872]}

data['group'] = None
for g in groups.items():
    for i in g[1]:
        data.ix[data.id==i, 'group'] = g[0]

In [75]:

# data.ix[data.id==919]

# TF-IDF

In [76]:
import nltk
import re
import string
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer


class Matcher:
    def __init__(self, num_match_returned=5):
        self.num_match_returned = num_match_returned
        self.stpwds = nltk.corpus.stopwords.words("french")
        self.vectorizer = None
        self.dtm = None
        self.questions_id = None
        # rajouter des mots commes les, docteur, ...

    def tokenize(self, content):
        if content is None:
            return None
        else:
            #remove http links
            content = re.sub(r'http\S+', '', content)
            # remove formatting
            content = re.sub("\s+", " ", content)
            # convert to lower case
            content = content.lower()
            # Remove accent
            content = unidecode(content)
            # remove punctuation (preserving intra-word dashes)
            # content = "".join(letter for letter in content if letter not in punct)
            punct = string.punctuation.replace("-", "")
            regex = re.compile('[%s]' % re.escape(punct))
            content = regex.sub(' ', content)
            content = re.sub("[^a-zA-Z]", " ", content)
            # remove dashes attached to words but that are not intra-word
            content = re.sub("[^[:alnum:]['-]", " ", content)
            content = re.sub("[^[:alnum:][-']", " ", content)

            # remove extra white space
            content = re.sub(" +", " ", content)
            # remove leading and trailing white space
            content = content.strip()
            # tokenize
            tokens = content.split(" ")
            # remove stopwords
            tokens = [token for token in tokens if token not in self.stpwds and len(token) > 2]
            return tokens
        
    def tokenize_dataset(self, data):
        tokens = []
        l = data.txt
        if isinstance(l,str):
            l = [l]
        tokens = [self.tokenize(q) for q in l]
        return tokens

    def train(self, X):
        tokens = self.tokenize_dataset(X)
        voc = list(set([i for sublist in tokens if sublist is not None for i in sublist]))
        join_tokens = [" ".join(i) for i in tokens]
        self.vectorizer = CountVectorizer(vocabulary=voc)
        self.dtm = self.vectorizer.fit_transform(join_tokens)
        return True

    def transform(self, X):
        if self.vectorizer is None:
            self.train(X)
        query_tokens = self.tokenize(query)
        join_tokens = " ".join(query_tokens)
        transf = self.vectorizer.transform([join_tokens])
        return transf

In [77]:
m = Matcher()
m.train(data)
X = m.dtm

# k-means

In [182]:
from sklearn.cluster import KMeans
init = []
unique_labels = [i for i in data.group.unique() if i is not None]
for u in unique_labels:
    init.append(X.toarray()[data.group==u,:].mean(axis=0))
init = np.array(init)

est = KMeans(n_clusters=4)
est.fit(X)
labels = est.labels_
unique_labels = ['k-means '+str(i) for i in np.unique(labels)]
labels2 = np.zeros(labels.shape).astype(object)
for u in np.unique(labels):
    labels2[labels==u] = 'k-means '+str(u)
np.unique(labels2)

array(['k-means 0', 'k-means 1', 'k-means 2', 'k-means 3'], dtype=object)

In [183]:
df = pd.DataFrame([data.group[~pd.isnull(data.group)].values,labels2[~pd.isnull(data.group)]]).transpose()
df.columns = ['true','pred']
df.groupby(by=['pred','true']).size()

pred       true             
k-means 0  administratif        8
           diagnostique         7
           médicaments          9
           pratique médicale    6
k-means 1  administratif        2
           médicaments          1
           pratique médicale    3
k-means 2  pratique médicale    1
k-means 3  diagnostique         3
dtype: int64

In [184]:
a = df.groupby(by=['pred','true']).size()
print('>>> Evaluation of K-means clusters')
for i in df.pred.unique():
    score = a.ix[i].max()/(1+a.ix[i].sum())
    print("- {}, Score: {}".format(i, score))
    print("Of {} samples put in that cluster, {} are of type {}".format(a.ix[i].sum(), a.ix[i].max(), a.ix[i].argmax()))

>>> Evaluation of K-means clusters
- k-means 0, Score: 0.2903225806451613
Of 30 samples put in that cluster, 9 are of type médicaments
- k-means 2, Score: 0.5
Of 1 samples put in that cluster, 1 are of type pratique médicale
- k-means 1, Score: 0.42857142857142855
Of 6 samples put in that cluster, 3 are of type pratique médicale
- k-means 3, Score: 0.75
Of 3 samples put in that cluster, 3 are of type diagnostique


<function print>